In [19]:
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch
from glob import glob
import pandas as pd
import numpy as np

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aumpa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Load model
model_name = "facebook/bart-large-mnli"
device = 0 if torch.cuda.is_available() else 'cpu'
device

0

In [4]:
def load_model(device):
    theme_classifier = pipeline(
        "zero-shot-classification",
        model = model_name,
        device=device
    )
    return theme_classifier

In [5]:
theme_classifier = load_model(device)

c:\Users\aumpa\anaconda3\envs\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
theme_list = ["friendship","hope","sacrifice","battle","self developement","betrayal","love","dialogue"]

In [7]:
theme_classifier(
    "I gave him a right hook than a left jab",
    theme_list,
    multi_label=True
)

{'sequence': 'I gave him a right hook than a left jab',
 'labels': ['battle',
  'self developement',
  'sacrifice',
  'betrayal',
  'hope',
  'dialogue',
  'friendship',
  'love'],
 'scores': [0.8533867001533508,
  0.5064211487770081,
  0.11604731529951096,
  0.057055965065956116,
  0.04676217585802078,
  0.0178579930216074,
  0.0017236591083928943,
  0.0010477364994585514]}

In [16]:
#load data
files = glob('../data/Subtitles/*.ass')

In [ ]:
with open(files[0],'r') as file:
    lines= file.readlines()
    lines = lines[27:]
    lines = [ ",".join(line.split(',')[9:]) for line in lines]
    
lines[:5]
    

In [ ]:
lines = [line.replace('\\N',' ') for line in lines]
lines[:5]

In [ ]:
int(files[0].split('-')[-1].split('.')[0].strip())

In [8]:
def load_subtitles_dataset(dataset_path):
    subtitles_paths = glob(dataset_path+'/*.ass')

    scripts=[]
    episode_num=[]

    for path in subtitles_paths:

        #Read Lines
        with open(path,'r') as file:
            lines = file.readlines()
            lines = lines[27:]
            lines =  [ ",".join(line.split(',')[9:])  for line in lines ]
        
        lines = [ line.replace('\\N',' ') for line in lines]
        script = " ".join(lines)

        episode = int(path.split('-')[-1].split('.')[0].strip())

        scripts.append(script)
        episode_num.append(episode)

    df = pd.DataFrame.from_dict({"episode":episode_num, "script":scripts })
    return df

In [9]:
dataset_path = '../data/Subtitles'
df = load_subtitles_dataset(dataset_path=dataset_path)

In [10]:
df.head()

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [11]:
script = df.iloc[0]['script']
script

'A long time ago, a powerful demon fox appeared with nine tails.\n With its powerful tails,\n it could smash mountains and create tidal waves.\n A band of Ninjas rose to defend their village from attack.\n We have to wait until the Fourth Hokage gets here!\n We can\'t let it get any closer to our village!\n One great Ninja was able to imprison the monster,\n but died in the process.\n This Ninja was known as… the Fourth Hokage.\n Naruto!\n Why did you do such a thing?!\n You\'re really gonna get it this time!\n I don\'t care!\n You know your problem?\n You can\'t do the things I do!\n Only I can do this!\n I\'m better than all of you! Believe it!\n There\'s a problem, sir!\n Lord Hokage!\n What is it?\n Did that Naruto do something again?\n Yes. He climbed onto the Mountainside Images…\n And he vandalized and graffitied all over them!\n Wait!\n Ha ha…\n Why should I?\n Hey, Naruto!\n How did you suddenly get here, lruka Sensei?\n The question is what are you doing here when you should 

In [12]:
script_sentences = sent_tokenize(script)
script_sentences[:3]

['A long time ago, a powerful demon fox appeared with nine tails.',
 'With its powerful tails,\n it could smash mountains and create tidal waves.',
 'A band of Ninjas rose to defend their village from attack.']

In [14]:
# Batch Sentence
sentence_batch_size=20
script_batches = []
for index in range(0,len(script_sentences),sentence_batch_size):
    sent = " ".join(script_sentences[index:index+sentence_batch_size])
    script_batches.append(sent)

script_batches[:2]

["A long time ago, a powerful demon fox appeared with nine tails. With its powerful tails,\n it could smash mountains and create tidal waves. A band of Ninjas rose to defend their village from attack. We have to wait until the Fourth Hokage gets here! We can't let it get any closer to our village! One great Ninja was able to imprison the monster,\n but died in the process. This Ninja was known as… the Fourth Hokage. Naruto! Why did you do such a thing?! You're really gonna get it this time! I don't care! You know your problem? You can't do the things I do! Only I can do this! I'm better than all of you! Believe it! There's a problem, sir! Lord Hokage! What is it? Did that Naruto do something again?",
 'Yes. He climbed onto the Mountainside Images…\n And he vandalized and graffitied all over them! Wait! Ha ha…\n Why should I? Hey, Naruto! How did you suddenly get here, lruka Sensei? The question is what are you doing here when you should be in class now? Now listen, Naruto. You failed t

In [15]:
theme_output = theme_classifier(
    script_batches[:2],
    theme_list,
    multi_label=True
)

In [16]:
theme_output

[{'sequence': "A long time ago, a powerful demon fox appeared with nine tails. With its powerful tails,\n it could smash mountains and create tidal waves. A band of Ninjas rose to defend their village from attack. We have to wait until the Fourth Hokage gets here! We can't let it get any closer to our village! One great Ninja was able to imprison the monster,\n but died in the process. This Ninja was known as… the Fourth Hokage. Naruto! Why did you do such a thing?! You're really gonna get it this time! I don't care! You know your problem? You can't do the things I do! Only I can do this! I'm better than all of you! Believe it! There's a problem, sir! Lord Hokage! What is it? Did that Naruto do something again?",
  'labels': ['dialogue',
   'betrayal',
   'battle',
   'sacrifice',
   'self developement',
   'hope',
   'friendship',
   'love'],
  'scores': [0.980073869228363,
   0.9396898746490479,
   0.8546878099441528,
   0.7349806427955627,
   0.6545627117156982,
   0.199098035693168

In [17]:
#wrangle output
themes = {}
for output in theme_output:
    for label,score in zip(output['labels'],output['scores']):
        if label not in themes:
            themes[label] = []
        themes[label].append(score)

In [18]:
themes

{'dialogue': [0.980073869228363, 0.9370124340057373],
 'betrayal': [0.9396898746490479, 0.6457233428955078],
 'battle': [0.8546878099441528, 0.6581301689147949],
 'sacrifice': [0.7349806427955627, 0.6258822083473206],
 'self developement': [0.6545627117156982, 0.8029861450195312],
 'hope': [0.19909803569316864, 0.20424054563045502],
 'friendship': [0.05922286584973335, 0.08603306859731674],
 'love': [0.040261685848236084, 0.028020532801747322]}

In [21]:
def get_themes_inference(script):
    script_sentences = sent_tokenize(script)

    # Batch Sentence
    sentence_batch_size=20
    script_batches = []
    for index in range(0,len(script_sentences),sentence_batch_size):
        sent = " ".join(script_sentences[index:index+sentence_batch_size])
        script_batches.append(sent)
    
    # Run Model
    theme_output = theme_classifier(
        script_batches[:2],
        theme_list,
        multi_label=True
    )

    # Wrangle Output 
    themes = {}
    for output in theme_output:
        for label,score in zip(output['labels'],output['scores']):
            if label not in themes:
                themes[label] = []
            themes[label].append(score)

    themes = {key: np.mean(np.array(value)) for key,value in themes.items()}

    return themes

In [22]:
df.shape

(218, 2)

In [23]:
df = df.head(2)

In [24]:
df

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."


In [25]:
output_themes = df['script'].apply(get_themes_inference)

In [26]:
output_themes

0    {'dialogue': 0.9585431516170502, 'betrayal': 0...
1    {'dialogue': 0.9606055617332458, 'sacrifice': ...
Name: script, dtype: object

In [27]:
themes_df=pd.DataFrame(output_themes.to_list())

In [28]:
themes_df

,dialogue,betrayal,battle,sacrifice,self developement,hope,friendship,love
0,0.958543,0.792707,0.756409,0.680431,0.728774,0.201669,0.072628,0.034141
1,0.960606,0.429943,0.684844,0.570702,0.419744,0.154534,0.046260,0.173260


In [29]:
df[themes_df.columns] = themes_df
df

,episode,script,dialogue,betrayal,battle,sacrifice,self developement,hope,friendship,love
0,1,"A long time ago, a powerful demon fox appeared...",0.958543,0.792707,0.756409,0.680431,0.728774,0.201669,0.072628,0.034141
1,2,"C'mon!\n Running like a fugitive,\n Being chas...",0.960606,0.429943,0.684844,0.570702,0.419744,0.154534,0.046260,0.173260
